In [1]:
from typing import List, Optional
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext
from langchain_community.retrievers import WikipediaRetriever
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv()


True

Flow

Input topic -> Topic Expansion (list) \
Topic Expansion -> Research on relevant ropics (dict) \
Research on relevant topics -> Generate Outline (str) \
Generate Outline -> Generate Article (str) \


## Topics Expansion


In [2]:
class RelatedTopics(BaseModel):
    topics: List[str] = Field(description="A list of related topics with maximum of 3.")


# Create agent
topic_explorer_agent = Agent(
    "openai:gpt-4o-mini",
    result_type=RelatedTopics,
    system_prompt=(
        "You are an experienced researcher. Given a research topic, return a list of relevant topics to research on."
        "Avoid returning topics that are too broad or too specific. "
        "The list of topics will then use to research on wikipedia to find relevant information. "
    ),
)


async def gen_related_topics(topic: str):
    related_topics = await topic_explorer_agent.run(topic)
    # related_topics_str = ", ".join(related_topics.data.topics)
    return related_topics.data.topics


topic = "AI Agents in Workforce."

related_topics = await gen_related_topics(topic)
print(related_topics)

['AI in Recruitment', 'Impact of AI on Labor Markets', 'Ethical Considerations of AI in the Workplace']


## Generate Outline


In [29]:
wiki_retriever = WikipediaRetriever(top_k_results=1)

wiki_retriever_results = await wiki_retriever.abatch(related_topics)
wiki_docs = wiki_retriever_results

[[Document(metadata={'title': 'Applications of artificial intelligence', 'summary': 'Artificial intelligence (AI) has been used in applications throughout industry and academia. In a manner analogous to electricity or computers, AI serves as a general-purpose technology. AI programes emulate perception and understanding, and are designed to adapt to new information and new situations. Machine learning has been used for various scientific and commercial purposes including language translation, image recognition, decision-making, credit scoring, and e-commerce.', 'source': 'https://en.wikipedia.org/wiki/Applications_of_artificial_intelligence'}, page_content="Artificial intelligence (AI) has been used in applications throughout industry and academia. In a manner analogous to electricity or computers, AI serves as a general-purpose technology. AI programes emulate perception and understanding, and are designed to adapt to new information and new situations. Machine learning has been used 

In [ ]:
from IPython.display import display, Markdown

display(Markdown(article.data.as_str))

#Local AI Adoption: Transforming Communities through Artificial Intelligence

## Understanding Local AI Adoption

Artificial intelligence (AI) adoption at the local level refers to the integration and utilization of AI technologies by local governments, businesses, and communities to improve public services, enhance decision-making, and foster economic growth. With the rise of AI capabilities, local entities are increasingly exploring how to leverage AI's potential to address community-specific challenges, streamline operations, and better serve constituents. This article provides an overview of local AI adoption, its benefits, challenges, and community-driven initiatives that are shaping its landscape.



## The Benefits of Local AI Adoption

AI offers transformative benefits for local communities across various sectors. Key advantages include:

1. **Enhanced Decision-Making**: AI enables the analysis of vast amounts of data, providing valuable insights for informed policy making.
2. **Improved Service Delivery**: Local governments can use AI to automate routine tasks and enhance service responsiveness, ultimately leading to better citizen experiences.
3. **Community Development**: AI technologies drive community development by fostering innovation, improving resource allocation, and increasing operational efficiencies.
4. **Economic Growth**: By facilitating new business models and optimizing systems, AI adoption can stimulate job creation and economic development in local economies.
5. **Personalized Community Engagement**: AI helps tailor communication with residents, allowing local authorities to better meet specific needs and preferences.

### Insights into Community Development

 AI's ability to analyze and interpret data offers profound insights into community trends, needs, and preferences. For instance, local governments can monitor traffic patterns or evaluate public health data to implement changes that enhance overall community well-being.

## Challenges to Local AI Adoption

Despite the numerous advantages, local AI adoption is not without challenges. Key hurdles include:

1. **Lack of Awareness**: Many local governments and organizations face a knowledge gap regarding AI technologies and their potential benefits.
2. **Budget Constraints**: Limited financial resources can hinder investments in AI infrastructure and expertise, curtailing the ability to adopt these technologies.
3. **Regulatory Barriers**: Complex and evolving AI regulations at the national and local levels can impede the adoption process, leaving organizations wary of compliance risks.
4. **Talent Shortages**: The demand for skilled professionals in AI is high, but many local governments struggle to recruit and retain experts.
5. **Public Perception and Acceptance**: There may be concerns regarding privacy, bias, and the ethical implications of AI, impacting community buy-in and support for AI initiatives.

### Navigating Regulatory Challenges

 Local regulations can significantly affect AI implementation. Authorities must consider compliance with both local and federal laws governing data usage, privacy protection, and algorithmic accountability. Establishing clear guidelines can enable smooth AI adoption while addressing community concerns.

## Community-Driven AI Initiatives

Community-driven initiatives in AI involve local residents in the development and deployment of AI technologies. This collaborative approach ensures that the voices of those affected are included in decision-making processes, leading to more tailored solutions that genuinely reflect community needs. Examples of community-driven AI initiatives include:

1. **Participatory Design Workshops**: Engaging community members in the design process of AI solutions to meet local needs and preferences.
2. **Open-Source Development**: Encouraging collaboration among local developers and stakeholders to share knowledge, tools, and data, thereby cultivating a supportive innovation ecosystem.
3. **Predictive Analytics for Public Safety**: Harnessing AI to analyze historical crime data and help law enforcement agencies allocate resources effectively, with community input on desired approaches to policing.

### Empowering Local Voices

 By empowering communities to have a say in AI projects, local governments can foster trust and minimize resistance. Initiatives like town hall meetings, surveys, and focus groups allow for two-way communication between local authorities and citizens, ensuring AI solutions align with community values.

## Conclusion

As local entities navigate the evolving landscape of AI adoption, balancing benefits with challenges is vital. Embracing community participation and fostering a supportive environment will be crucial for the successful implementation of AI technologies. By navigating the regulatory complexities, investing in education and training, and prioritizing ethical considerations, local governments can harness AI's full potential to enhance services, drive innovation, and contribute to the overall prosperity of their communities.



## Future Trends in Local AI Adoption

As we look ahead, several trends are likely to shape the future of AI adoption at the local level:

1. **Increased Collaboration**: Local governments will likely collaborate with tech companies and research institutions to leverage expertise and resources for AI initiatives.
2. **Focus on Ethical AI**: There will be greater emphasis on developing ethical AI guidelines to ensure responsible use of technology and to build public trust.
3. **Smart City Initiatives**: More local governments will integrate AI into their smart city projects to optimize urban infrastructure and public services.
4. **Training Programs**: Expect the emergence of training programs aimed at equipping local officials and staff with the necessary skills for effective AI integration.



## Research on the topics


In [2]:
from tavily import AsyncTavilyClient


class TavilySearch:
    def __init__(self, max_results: int = 3):
        TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
        self.tavily_client = AsyncTavilyClient(api_key=TAVILY_API_KEY)
        self.max_results = max_results

    async def asearch(self, query: str) -> dict:
        results = {}
        response = await self.tavily_client.search(query, max_results=self.max_results)
        search_results = response["results"]
        for r in search_results:
            source = {r["url"]: r["content"]}
            results.update(source)
        return results

    async def abatch_search(self, queries: list[str]) -> dict:
        search_results = {}
        for query in queries:
            results = await self.asearch(query)
            search_results.update(results)
        return search_results

    @staticmethod
    def results_to_str(search_results: dict, max_len: int = 50_000) -> str:
        return "\n\n##########\n".join(
            f"URL:\n {url}\n\nContent:\n{content}"
            for url, content in search_results.items()
        )[:max_len]


# Usage
tavily_search = TavilySearch()
search_results = await tavily_search.abatch_search(related_topics)
search_results_str = TavilySearch.results_to_str(search_results)

print(search_results_str)

NameError: name 'related_topics' is not defined

## Write Article


In [41]:
## Generate Outline
class Subsection(BaseModel):
    title: str = Field(description="The title of the subsection.")
    content: str = Field(description="The content of the subsection.")


class Section(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")
    subtitles: List[Subsection] = Field(description="A list of subsections.")

    @property
    def as_str(self):
        subsections = "\n\n".join(
            f"### {s.title}\n\n {s.content}" for s in self.subtitles
        )
        return f"## {self.title}\n\n{self.content}\n\n{subsections}"


class Outline(BaseModel):
    title: str = Field(description="The title of the article.")
    sections: List[Section]

    @property
    def as_str(self):
        sections = "\n\n".join(f"{s.as_str}" for s in self.sections)
        return f"# {self.title}\n\n{sections}"


In [52]:
# Generate Article
gen_article_agent = Agent(
    "openai:gpt-4o-mini",
    result_type=Outline,
    deps_type=Optional[Outline],
    system_prompt=(
        "You are an experienced researcher."
        "Based on the resouces gathered so far, generate an article based on the outline."
        "Use only the information provided gathered anf cite the url of the source for each section."
        "Make sure the article is informative and cohesive."
    ),
)


example_article = await gen_article_agent.run(
    search_results_str,
    result_type=Outline,
)


## Chaining everything together


In [58]:
from pydantic_ai import Tool

gen_article_agent = Agent(
    "openai:gpt-4o-mini",
    result_type=Outline,
    deps_type=Optional[Outline],
    system_prompt=(
        "You are an experienced researcher, given a research topic, use gen_related_topics to find more relevant topics."
        "Use tavily_search.abatch_search tool to gather relevant information with the relevant topics."
        "Based on the resouces gathered, generate an article based on the outline."
        "Use only the information provided gathered anf cite the url of the source for each section."
        "Make sure the article is informative and cohesive."
    ),
    tools=[Tool(tavily_search.abatch_search), Tool(gen_related_topics)],
)

article = await gen_article_agent.run("local AI adoption")

In [ ]:
from IPython.display import display, Markdown

display(Markdown(article.data.as_str))

## Result

Based on the research results, update the initial outline.


In [ ]:
from IPython.display import display, Markdown

display(Markdown(example_article.data.as_str))

In [ ]:
article.all_messages()